In [1]:
%run notes_events_network_join.ipynb

Using device: cuda

NVIDIA GeForce GTX 1060 6GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


# Hyperparameters experiment
We train and eval the model varying the following:

* cohorts: unbalanced vs balanced
* optimizers: Adam vs SGD
* learning rates
* epochs
* number of samples

In [2]:
cohorts = ['unbalanced', 'balanced']
optimizers = [torch.optim.Adam, torch.optim.SGD]
learning_rates = [0.0001, 0.001, 0.01]
epochs = [5, 10, 15]
samples = [1000, 5000, 0]

results = np.empty(shape=(len(cohorts), len(optimizers), len(learning_rates), len(epochs), len(samples)), dtype='object')
criterion = nn.BCELoss()

for c, cohort in enumerate(cohorts):
    for o, optim in enumerate(optimizers):
        for l, learning_rate in enumerate(learning_rates):
            for e, n_epochs in enumerate(epochs):
                for s, n_samples in enumerate(samples):
                    model, optimizer = create_model_and_optimizer()
                    optimizer = optim(model.parameters(), lr=learning_rate)
                    print ("Training for:\n")
                    print ("Cohort        \t= ", cohort)
                    print ("Optimizer     \t= ", optimizer)
                    print ("Learning rate \t= ", learning_rate)
                    print ("No. of epochs \t= ", n_epochs)
                    print ("No. of samples\t= ", n_samples)
                    print ('---------------')
                    model_filename = cohort + '-' + str(o) + '-' + str(learning_rate) + '-' + str(n_epochs) + '-' + str(n_samples) + '.pt'

                    train_loader, val_loader = get_unbalanced_dataloaders(n_samples) if cohort=='unbalanced' else get_balanced_dataloaders(n_samples)                    
                    p, r, f, roc_auc = train_and_eval(model, train_loader, val_loader, n_epochs, model_filename)
                    results[c,o,l,e,s] = [p, r, f, roc_auc]
                    print ('---------------\n\n')


=0.4178662896156311
Batch : 10, 20, 30, 40, 50, 60, 70, Epoch 10: curr_epoch_loss=0.42086124420166016
Batch : 10, 20, 30, 40, 50, 60, 70, Epoch 11: curr_epoch_loss=0.418766587972641
Batch : 10, 20, 30, 40, 50, 60, 70, Epoch 12: curr_epoch_loss=0.4168277680873871
Batch : 10, 20, 30, 40, 50, 60, 70, Epoch 13: curr_epoch_loss=0.4196009039878845
Batch : 10, 20, 30, 40, 50, 60, 70, Epoch 14: curr_epoch_loss=0.41362619400024414
Model Training time: 341.4437174797058
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Learning rate: 0.001
Model Training time: 341.4437174797058
Precision =  0.0
Recall    =  0.0
F1        =  0.0
ROC AUC   =  0.6524737881358288
0.0 	 0.0 	 0.0 	 0.6524737881358288
---------------


Training for:

Cohort        	

IndexError: index 1965 is out of bounds for axis 0 with size 1965

## Saving of the results

In [3]:
now = datetime.datetime.now()
r = {}
r['cohorts'] = cohorts
r['optimizers'] = optimizers
r['learning_rates'] = learning_rates
r['epochs'] = epochs
r['samples'] = samples
r['results'] = results
pickle.dump( r, open("hyperparam-exp-" + now.strftime("%Y%m%d-%H%M") +".p", "wb" ) )